In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
import joblib

In [ ]:
#path of file
excel_file = "./EmissionsDataset.xlsx"

years = range(2010, 2017)

In [ ]:
years[2]

In [ ]:
df_1 = pd.read_excel(excel_file, sheet_name=f'{years[0]}_Detail_Commodity')
df_1.head()

In [ ]:
df_2 = pd.read_excel(excel_file, sheet_name=f'{years[0]}_Detail_Industry')
df_2.head()

In [ ]:
all_data = []

In [ ]:
for year in years:
    try:
        df_com = pd.read_excel(excel_file, sheet_name=f'{year}_Detail_Commodity')
        df_ind = pd.read_excel(excel_file, sheet_name=f'{year}_Detail_Industry')
        
        df_com['Source'] = 'Commodity'
        df_ind['Source'] = 'Industry'
        df_com['Year'] = df_ind['Year'] = year
        
        df_com.columns = df_com.columns.str.strip()
        df_ind.columns = df_ind.columns.str.strip()

        df_com.rename(columns={
            'Commodity Code': 'Code',
            'Commodity Name': 'Name'
        }, inplace=True)
        
        df_ind.rename(columns={
            'Industry Code': 'Code',
            'Industry Name': 'Name'
        }, inplace=True)
        
        all_data.append(pd.concat([df_com, df_ind], ignore_index=True))
        
    except Exception as e:
        print(f"Error processing year {year}: {e}")


In [ ]:
all_data[3]

In [ ]:
len(all_data)

In [ ]:
df = pd.concat(all_data, ignore_index=True)
df.head(10)

In [ ]:
len(df)

In [ ]:
df.columns

In [ ]:
df.isnull().sum()

In [ ]:
# As there is no data avaialble in Unnamed coulmn so we will drop the column
df.drop(columns=['Unnamed: 7'],inplace=True)

In [ ]:
df.columns

In [ ]:
print(df.info())   # Checking data types and non-null counts 

In [ ]:
df.describe().T # Checking summary statistics 

In [ ]:

df.isnull().sum() # Checking for null values in each column 

In [ ]:
# Check categorical variables
print(df['Substance'].value_counts())

In [ ]:
print(df['Unit'].value_counts()) # Checking unique values in 'Unit' with count 

In [ ]:
print(df['Source'].value_counts()) # Checking unique values in 'Source' with count 

In [ ]:
df['Substance'].unique() # Checking unique values in 'Substance' 

In [ ]:
substance_map={'carbon dioxide':0, 'methane':1, 'nitrous oxide':2, 'other GHGs':3} # Mapping substances to integers 

In [ ]:
df['Substance']=df['Substance'].map(substance_map) 

In [ ]:
df['Substance'].unique() # Checking unique values in 'Substance' 

In [ ]:
print(df['Unit'].unique()) # Checking unique values in 'Unit' 

In [ ]:
unit_map={'kg/2018 USD, purchaser price':0, 'kg CO2e/2018 USD, purchaser price':1} # Mapping units to integers 

In [ ]:
df['Unit']=df['Unit'].map(unit_map)
print(df['Unit'].unique()) # Checking unique values in 'Unit' 

In [ ]:
print(df['Source'].unique()) # Checking unique values in 'Source' 

In [ ]:
source_map={'Commodity':0, 'Industry':1} # Mapping sources to integers 
df['Source']=df['Source'].map(source_map)   # applying the mapping to 'Source' column 
print(df['Source'].unique()) # Checking unique values in 'Source' 

In [ ]:
df.info() # Checking data types and non-null counts after mapping 

In [ ]:
df.Code.unique() # Checking unique values in 'Code' df['Code']

In [ ]:
df.Name.unique() # Checking unique values in 'Name' 
 

In [ ]:
len(df.Name.unique()) # Checking number of unique values in 'Name'

In [ ]:
top_emitters = df[['Name', 'Supply Chain Emission Factors with Margins']].groupby('Name').mean().sort_values(
    'Supply Chain Emission Factors with Margins', ascending=False).head(10) 

# Resetting index for better plotting
top_emitters = top_emitters.reset_index()

In [ ]:
top_emitters

In [ ]:
# Plotting the top 10 emitting industries


plt.figure(figsize=(10,6))
# Example: Top emitting industries (already grouped)
sns.barplot(
    x='Supply Chain Emission Factors with Margins',
    y='Name',
    data=top_emitters,
    hue='Name',
    palette='viridis'  # Use 'Blues', 'viridis', etc., for other color maps
)

# Add ranking labels (1, 2, 3...) next to bars
for i, (value, name) in enumerate(zip(top_emitters['Supply Chain Emission Factors with Margins'], top_emitters.index), start=1):
    plt.text(value + 0.01, i - 1, f'#{i}', va='center', fontsize=11, fontweight='bold', color='black')

plt.title('Top 10 Emitting Industries', fontsize=14, fontweight='bold') # Title of the plot 
plt.xlabel('Emission Factor (kg CO2e/unit)') # X-axis label
plt.ylabel('Industry') # Y-axis label
plt.grid(axis='x', linestyle='--', alpha=0.6) # Adding grid lines for better readability
plt.tight_layout() # Adjust layout to prevent overlap

plt.show()

In [ ]:

df.drop(columns=['Name','Code','Year'], inplace=True) 

In [ ]:
df.head(1)

In [ ]:
df.shape

In [ ]:
df.columns

In [ ]:
X = df.drop(columns=['Supply Chain Emission Factors with Margins']) # Feature set excluding the target variable
y = df['Supply Chain Emission Factors with Margins'] # Target variable 
X.head()

In [ ]:
y.head()

In [ ]:
# Count plot for Substance
plt.figure(figsize=(6, 3))
sns.countplot(x=df["Substance"])
plt.title("Count Plot: Substance")
plt.xticks()
plt.tight_layout()
plt.show()

In [ ]:
# Count plot for Unit
plt.figure(figsize=(6, 3))
sns.countplot(x=df["Unit"])
plt.title("Count Plot: Unit")
plt.tight_layout()
plt.show()


In [ ]:
# Count plot for Source
plt.figure(figsize=(6, 4))
sns.countplot(x=df["Source"])
plt.title("Count Plot: Source (Industry vs Commodity)")
plt.tight_layout()
plt.show()

In [ ]:
df.columns

In [ ]:
df.select_dtypes(include=np.number).corr() # Checking correlation between numerical features 

In [ ]:
df.info() # Checking data types and non-null counts after mapping 

In [ ]:
# Correlation matrix 
plt.figure(figsize=(12, 8))
sns.heatmap(df.select_dtypes(include=np.number).corr(), annot=True, cmap="coolwarm")
plt.title("Correlation Heatmap")
plt.show()

In [ ]:
X.describe().T

In [ ]:
#normalize
scaler=StandardScaler()
X_scaled=scaler.fit_transform(X)


In [ ]:
np.round(X_scaled.mean()),np.round(X_scaled.std())

In [ ]:
X.shape

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42) # Splitting data into training and testing sets 

In [ ]:
X_train.shape

In [ ]:
X_test.shape

In [ ]:
 # Initializing Random Forest Regressor 
RF_model = RandomForestRegressor(random_state=42)

In [ ]:
RF_model.fit(X_train, y_train) # Fitting the model on training data 

In [ ]:
RF_y_pred = RF_model.predict(X_test) # Making predictions on the test set 
RF_y_pred[:20]

In [ ]:
RF_mse = mean_squared_error(y_test, RF_y_pred) # Calculating Mean Squared Error (MSE)
RF_rmse = np.sqrt(RF_mse) # Calculating Root Mean Squared Error (RMSE)
# Calculating R² score
RF_r2 = r2_score(y_test, RF_y_pred)

print(f'RMSE: {RF_rmse}')
print(f'R² Score: {RF_r2}')

In [ ]:
from sklearn.linear_model import LinearRegression # Importing Linear Regression model 
LR_model = LinearRegression() # Initializing Linear Regression model
# Fitting the Linear Regression model on training data

LR_model.fit(X_train, y_train)

LR_y_pred = LR_model.predict(X_test) # Making predictions on the test set using Linear Regression model 


LR_mse = mean_squared_error(y_test, LR_y_pred) # Calculating Mean Squared Error (MSE) for Linear Regression model
LR_rmse = np.sqrt(LR_mse) # Calculating Root Mean Squared Error (RMSE) for Linear Regression model 
LR_r2 = r2_score(y_test, LR_y_pred) # Calculating R² score for Linear Regression model 

print(f'RMSE: {LR_rmse}')
print(f'R² Score: {LR_r2}')

In [ ]:
# Hyperparameter tuning for Random Forest Regressor using GridSearchCV 
# Define the parameter grid for hyperparameter tuning 
param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5]
}

# Perform grid search with cross-validation to find the best hyperparameters 
grid_search = GridSearchCV(RandomForestRegressor(random_state=42), param_grid, cv=3, n_jobs=-1)

# Fit the grid search model on the training data 
grid_search.fit(X_train, y_train)

# Best model from grid search
best_model = grid_search.best_estimator_
print("Best Parameters:", grid_search.best_params_)


In [ ]:
# Use the best model to make predictions on the test set 
y_pred_best = best_model.predict(X_test)


HP_mse = mean_squared_error(y_test, y_pred_best)
HP_rmse = np.sqrt(HP_mse)
HP_r2 = r2_score(y_test, y_pred_best)

print(f'RMSE: {HP_rmse}')
print(f'R² Score: {HP_r2}')


In [ ]:
# Create a comparative DataFrame for all models
results = {
    'Model': ['Random Forest (Default)', 'Linear Regression', 'Random Forest (Tuned)'],
    'MSE': [RF_mse, LR_mse, HP_mse],
    'RMSE': [RF_rmse, LR_rmse, HP_rmse],
    'R2': [RF_r2, LR_r2, HP_r2]
}

# Create a DataFrame to compare the results of different models
comparison_df = pd.DataFrame(results)
print(comparison_df)

In [ ]:
# Create a directory to save the models if it doesn't exist 
!mkdir models 

In [ ]:
# Save model and encoders 
joblib.dump(best_model, 'models/LR_model.pkl')    # Save the best model 
joblib.dump(scaler, 'models/scaler.pkl') # Save the scaler used for normalization
